# QSM Reconstruction

In [ ]:
!run_2_qsm.py bids-new qsm-new --premade body --auto_yes --slurm a_barth general

In [ ]:
!for f in `find qsm-new/qsm_final -name '*.nii*'`; do sub="${f/*sub/sub}"; sub="${sub/_ses*/}"; ses="${f/*_ses/ses}"; ses="${ses/_*/}"; cp "${f}" "bids-new/${sub}/${ses}/extra_data/"; done

In [ ]:
!for f in `find . -wholename '*numpy_nibabel_twopass/mapflow/*/*echo-02*twopass.nii*'`; do sub="${f/*sub/sub}"; sub="${sub/_ses*/}"; ses="${f/*_ses/ses}"; ses="${ses/_*/}"; f2="${f/twopass1/twopass3}"; f2="${f2/echo-02/echo-04}"; ~/qsmxt/interfaces/nipype_interface_nonzeroaverage.py "${f}" "${f2}" "bids-new/${sub}/${ses}/extra_data/${sub}_${ses}_qsm_echo2-and-echo4.nii"; done

# Imports

In [1]:
import os
import glob
import nibabel as nib
import nilearn.image
import numpy as np
import enum
import json
import cv2
import scipy

# Locate data

In [2]:
bids_dir = "bids-new"

In [3]:
session_dirs = []
for json_path in sorted(glob.glob(os.path.join(bids_dir, "sub*", "ses*", "anat", "*echo-01*mag*json"))):
    with open(json_path, 'r') as json_file:
        json_data = json.load(json_file)
        if json_data['ProtocolName'] == "t2starME_qsm_tra_Iso1.4mm_INPHASE_bipolar_RUN_THIS_ONE":
            session_dirs.append(os.sep.join(os.path.split(json_path)[0].split(os.sep)[:-1]))
print(f"{len(session_dirs)} sessions found:")
for session_dir in session_dirs: print(session_dir)

18 sessions found:
bids-new/sub-z0034542/ses-20220715
bids-new/sub-z0186251/ses-20221107
bids-new/sub-z0237546/ses-20230508
bids-new/sub-z0445614/ses-20230510
bids-new/sub-z0705200/ses-20230104
bids-new/sub-z0755228/ses-20211108
bids-new/sub-z1167038/ses-20220315
bids-new/sub-z1181657/ses-20220315
bids-new/sub-z1262112/ses-20220314
bids-new/sub-z1472355/ses-20221222
bids-new/sub-z1568577/ses-20230510
bids-new/sub-z1728751/ses-20220328
bids-new/sub-z1778013/ses-20220715
bids-new/sub-z1818796/ses-20230313
bids-new/sub-z2007565/ses-20220715
bids-new/sub-z2904752/ses-20220826
bids-new/sub-z3171177/ses-20230313
bids-new/sub-z3278008/ses-20211109


In [6]:
qsm_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*qsm_echo2-and-echo4.*")) for session_dir in session_dirs), []))
seg_clean_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "sub*ses*segmentation_clean.*")) for session_dir in session_dirs), []))
t1_resampled_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*t1_tra*_resampled.nii*")) for session_dir in session_dirs), []))
t2s_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*t2starmap.nii*")) for session_dir in session_dirs), []))
mag_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "magnitude_combined.nii")) for session_dir in session_dirs), []))

t1_files = [t1_file.replace("_resampled", "") for t1_file in t1_resampled_files]
seg_files = [seg_clean_file.replace("_clean", "") for seg_clean_file in seg_clean_files]

extra_files = sum((glob.glob(os.path.join(session_dir, "extra_data", "*.nii*")) for session_dir in session_dirs), [])
ct_files = [extra_file for extra_file in extra_files if any(pattern in extra_file for pattern in ['_na_', '_Pelvis_']) and not any(pattern in extra_file for pattern in ['_t1_tra_', 'ATX', 'AXT', 'ROI', 'resliced', 'segmentation'])]

ct_seg_files_unclean = sum((glob.glob(ct_file.replace(".nii", "_segmentation.nii")) for ct_file in ct_files), [])
ct_seg_files = sum((glob.glob(ct_file.replace(".nii", "_segmentation_clean.nii")) for ct_file in ct_files), [])
ct_resliced_files = sum((glob.glob(ct_file.replace(".nii", "_resliced.nii")) for ct_file in ct_files), [])
ct_resliced_seg_files_unclean = sum((glob.glob(ct_file.replace(".nii", "_segmentation.nii")) for ct_file in ct_resliced_files), [])
ct_resliced_seg_files = sum((glob.glob(ct_file.replace(".nii", "_segmentation_clean.nii")) for ct_file in ct_resliced_files), [])

print(f"{len(ct_files)} CT images found.")
print(f"{len(ct_seg_files_unclean)} CT segmentations found.")
print(f"{len(ct_seg_files)} cleaned-up CT segmentations found.")
print(f"{len(ct_resliced_files)} resliced CT images found.")
print(f"{len(ct_resliced_seg_files_unclean)} resliced CT segmentations found.")
print(f"{len(ct_resliced_seg_files)} cleaned-up and resliced CT segmentations found.")
print(f"{len(qsm_files)} QSM images found.")
print(f"{len(mag_files)} magnitude images found.")
print(f"{len(t2s_files)} T2* maps found.")
print(f"{len(t1_files)} T1w files found.")
print(f"{len(t1_resampled_files)} resampled T1w files found.")
print(f"{len(seg_files)} GRE segmentations found.")
print(f"{len(seg_clean_files)} cleaned GRE segmentations found.")

18 CT images found.
18 CT segmentations found.
18 cleaned-up CT segmentations found.
18 resliced CT images found.
18 resliced CT segmentations found.
18 cleaned-up and resliced CT segmentations found.
18 QSM images found.
18 magnitude images found.
18 T2* maps found.
18 T1w files found.
18 resampled T1w files found.
18 GRE segmentations found.
18 cleaned GRE segmentations found.


# Clean up segmentations

In [ ]:
class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

for i in range(len(qsm_files)):
    # load files
    print(qsm_files[i], seg_files[i])
    qsm_nii = nib.load(qsm_files[i])
    seg_nii = nib.load(seg_files[i])

    # get image data
    qsm = qsm_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # separate prostate tissue values and then remove segmented area
    prostate_values = qsm[seg == SegType.PROSTATE.value]
    print("Prostate value:", prostate_values.mean())

    # remove values close to the mean
    seg[np.logical_and(
        seg == SegType.GOLD_SEED.value,
        np.logical_and(
            qsm < prostate_values.mean() + 1*prostate_values.std(),
            qsm > prostate_values.mean() - 1*prostate_values.std()
        )
    )] = 0

    #smoothed = scipy.ndimage.gaussian_filter((seg == SegType.GOLD_SEED.value) * 1.0, sigma=[2,2,2])
    #seg[SegType.GOLD_SEED.value] = 0
    #seg[smoothed > 0.12] = SegType.GOLD_SEED.value

    seg[seg == SegType.CALCIFICATION.value] = 0
    calc_mask = np.logical_and(seg == SegType.NO_LABEL.value, qsm < np.mean(prostate_values) - 2*np.std(prostate_values))
    calc_mask = scipy.ndimage.binary_opening(calc_mask)

    seg[seg == SegType.PROSTATE.value] = 0
    seg[seg == SegType.GOLD_SEED.value] = 1
    seg[calc_mask] = 2
    #seg[seg == SegType.CALCIFICATION.value] = 0
    #calc_mask = np.logical_and(seg == SegType.NO_LABEL.value, qsm < np.mean(prostate_values) - 4*np.std(prostate_values))
    #calc_mask = scipy.ndimage.binary_opening(calc_mask)
    #seg[calc_mask] = 2

    # save result using original file extension
    filename=f"{seg_files[i].replace('.nii', '_clean.nii')}"
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"Saved {filename}")

In [15]:
class SegType(enum.Enum):
    NO_LABEL = 0
    GOLD_SEED = 1
    CALCIFICATION = 2
    PROSTATE = 3

for i in range(len(ct_resliced_files)):
    # load files
    ct_nii = nib.load(ct_resliced_files[i])
    seg_nii = nib.load(ct_resliced_seg_files_unclean[i])

    # get image data
    ct = ct_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # separate prostate tissue values and then remove segmented area
    prostate_values = ct[seg == SegType.PROSTATE.value]
    seg[seg == SegType.PROSTATE.value] = 0

    # remove low values
    seg[np.logical_and(seg == SegType.GOLD_SEED.value, ct < np.mean(prostate_values) + 4*np.std(prostate_values))] = 0
    seg[np.logical_and(seg == SegType.CALCIFICATION.value, ct < np.mean(prostate_values) + 4*np.std(prostate_values))] = 0

    # create new custom calcification label    
    seg[seg == SegType.CALCIFICATION.value] = 0
    calc_mask = np.logical_and(seg == SegType.NO_LABEL.value, ct > np.mean(prostate_values) + 6*np.std(prostate_values))
    #calc_mask = scipy.ndimage.binary_opening(calc_mask)
    seg[calc_mask] = 2 # NEW VALUE

    # save result using original file extension
    filename=f"{ct_resliced_seg_files_unclean[i].replace('.nii', '_clean.nii')}"
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"PROSTATE={len(ct[seg == SegType.PROSTATE.value])}")
    print(f"CALCIFICATION={len(ct[seg == SegType.CALCIFICATION.value])}")
    print(f"GOLD_SEED={len(ct[seg == SegType.GOLD_SEED.value])}")
    print(f"Saved {filename}\n")

PROSTATE=0
CALCIFICATION=114674
GOLD_SEED=146
Saved bids-new/sub-z0034542/ses-20220715/extra_data/5_na_20220715092125_5_resliced_segmentation_clean.nii

PROSTATE=0
CALCIFICATION=141514
GOLD_SEED=153
Saved bids-new/sub-z0186251/ses-20221107/extra_data/5_na_20221107081460_5_resliced_segmentation_clean.nii

PROSTATE=0
CALCIFICATION=134611
GOLD_SEED=158
Saved bids-new/sub-z0237546/ses-20230508/extra_data/7_na_20230508102530_7_resliced_segmentation_clean.nii

PROSTATE=0
CALCIFICATION=137889
GOLD_SEED=163
Saved bids-new/sub-z0445614/ses-20230510/extra_data/5_16_pelvis_16_Pelvis_20230510090050_5_resliced_segmentation_clean.nii

PROSTATE=0
CALCIFICATION=98507
GOLD_SEED=159
Saved bids-new/sub-z0705200/ses-20230104/extra_data/5_11_pelvis_11_Pelvis_20230104113938_5_resliced_segmentation_clean.nii

PROSTATE=0
CALCIFICATION=144091
GOLD_SEED=148
Saved bids-new/sub-z0755228/ses-20211108/extra_data/Z0755228_20211109_Pelvis_2.0_HD_FoV_iMAR_20211109085844_5_resliced_segmentation_clean.nii

PROSTATE=0
CA

In [ ]:
class SegType(enum.Enum):
    NO_LABEL = 0
    GOLD_SEED = 1
    CALCIFICATION = 2
    PROSTATE = 3

for i in range(len(ct_files)):
    # load files
    ct_nii = nib.load(ct_files[i])
    seg_nii = nib.load(ct_seg_files[i])

    # get image data
    ct = ct_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # separate prostate tissue values and then remove segmented area
    prostate_values = ct[seg == SegType.PROSTATE.value]
    seg[seg == SegType.PROSTATE.value] = 0

    # remove low values
    seg[np.logical_and(seg == SegType.GOLD_SEED.value, ct < np.mean(prostate_values) + 4*np.std(prostate_values))] = 0
    seg[np.logical_and(seg == SegType.CALCIFICATION.value, ct < np.mean(prostate_values) + 4*np.std(prostate_values))] = 0

    # create new custom calcification label    
    seg[seg == SegType.CALCIFICATION.value] = 0
    calc_mask = np.logical_and(seg == SegType.NO_LABEL.value, ct > np.mean(prostate_values) + 1*np.std(prostate_values))
    calc_mask = scipy.ndimage.binary_opening(calc_mask)
    seg[calc_mask] = 2 # NEW VALUE

    # save result using original file extension
    filename=f"{ct_seg_files[i].replace('.nii', '_clean.nii')}"
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"PROSTATE={len(ct[seg == SegType.PROSTATE.value])}")
    print(f"CALCIFICATION={len(ct[seg == SegType.CALCIFICATION.value])}")
    print(f"GOLD_SEED={len(ct[seg == SegType.GOLD_SEED.value])}")
    print(f"Saved {filename}\n")

# Combined magnitude

In [ ]:
for session_dir in session_dirs:
    mag_images = sorted(glob.glob(os.path.join(session_dir, "anat", "*part-mag*nii*")))
    mag_images_even = [mag_images[i] for i in range(1, len(mag_images), 2)]
    nii = nib.load(mag_images_even[0])
    mag_4d = np.array([nib.load(mag_images_even[i]).get_fdata() for i in range(len(mag_images_even))])
    mag_combined = np.sqrt(np.sum(np.square(mag_4d), axis=0))
    nib.save(nib.Nifti1Image(mag_combined, header=nii.header, affine=nii.affine), os.path.join(session_dir, "extra_data", "magnitude_combined.nii"))

# Reslice T1w images

In [ ]:
for i in range(len(t1_files)):
    print(t1_files)
    t1_nii = nib.load(t1_files[i])
    mag_nii = nib.load(mag_files[i])
    t1_resampled = nilearn.image.resample_img(t1_nii, target_affine=mag_nii.affine, target_shape=mag_nii.header.get_data_shape())
    filename = t1_files[i].replace(".nii", "_resampled.nii")
    nib.save(t1_resampled, filename)

# T2* Maps

In [ ]:
for session_dir in session_dirs:
    !./make_t2starmaps.jl --magnitude {os.path.join(session_dir, "anat", "*echo-02*mag*.nii*")} {os.path.join(session_dir, "anat", "*echo-04*mag*.nii*")} --TEs "[0.0074,0.01482]" --output {os.path.join(session_dir, "extra_data", "t2starmap.nii")}
    #!./make_t2starmaps.jl --magnitude {os.path.join(bids_dir, session_dir)} bids-new/sub-z2072142/ses-20210128/anat/sub-z2072142_ses-20210128_run-01_echo-04_part-mag_MEGRE.nii --TEs [0.0074,0.01482] --output out2.nii

# SWI

In [ ]:
!julia -e "import Pkg; Pkg.add(Pkg.PackageSpec(url=\"https://github.com/korbinian90/CLEARSWI.jl\"))"

In [ ]:
#./make_swis.jl --phase bids-new/sub-z0034542/ses-20220715/anat/sub-z0034542_ses-20220715_run-01_echo-*_part-phase_MEGRE.nii --magnitude bids-new/sub-z0034542/ses-20220715/anat/sub-z0034542_ses-20220715_run-01_echo-*_part-mag_MEGRE.nii --TEs [0.00492,0.00738] --swi-out swi-out.nii --mip-out mip-out.nii

In [ ]:
for session_dir in session_dirs:
    !echo {session_dir}
    !./make_swis.jl \
        --magnitude {os.path.join(session_dir, "anat", "*echo-02*mag*.nii*")} {os.path.join(session_dir, "anat", "*echo-04*mag*.nii*")} \
        --phase {os.path.join(session_dir, "anat", "*echo-02*phase*.nii*")} {os.path.join(session_dir, "anat", "*echo-04*phase*.nii*")} \
        --TEs "[4.92,7.38]" \
        --swi-out {os.path.join(session_dir, "extra_data", "swi.nii")}